### IMPORTS

In [1]:
import json
from web3 import Web3

### LOAD SETTINGS & LATEST ABI FILE

In [2]:
with open('settings.json') as json_file:
    settings = json.load(json_file)

In [3]:
with open('latest.json') as json_file:
    latest = json.load(json_file)

### IPFS & BLOCKCHAIN GATEWAY CONNECTIONS

In [4]:
class gateway:
    def __init__(self, foo):
        self.host = foo['host']
        self.port = foo['port']

In [5]:
blockchain = gateway(settings['gateways']['blockchain'])

In [6]:
ipfs = gateway(settings['gateways']['ipfs'])

### BLOCKCHAIN PUBLIC/PRIVATE KEYS

In [7]:
class keypair:
    def __init__(self, foo):
        self.public = foo['public']
        self.private = foo['private']

In [8]:
keys = keypair(settings['keys'])

### CONNECT TO CHAIN VIA WEBSOCKET

In [9]:
web3 = Web3(Web3.WebsocketProvider('ws://' + blockchain.host + ':' + blockchain.port))

In [10]:
web3.isConnected()

True

### CONSTRUCT USABLE CONTRACTS

In [153]:
def contract(block):
    return web3.eth.contract(
        address = block['address'],
        abi = block['abi']
    )

In [183]:
class create_user_manager:
    
    # CONSTRUCT CONTRACT
    def __init__(self, block):
        self.contract = contract(block)
    
    # FETCH ALL USERS
    def everyone(self):
        return self.contract.functions.fetch_everyone().call()
    
    # REGISTER USER
    def register(self):
        return 'foo'

In [184]:
class create_token_manager:
    
    # CONSTRUCT CONTRACT
    def __init__(self, block):
        self.contract = contract(block)
    
    # FETCH ALL USERS
    def price(self):
        return self.contract.functions.price().call()

### CONTRACT REFERENCES

In [185]:
user_manager = create_user_manager(latest['usermanager'])

In [186]:
token_manager = create_token_manager(latest['tokenmanager'])

### TESTING

In [187]:
token_manager.price()

0

In [188]:
user_manager.everyone()

[]